In [1]:
# # Import packages
import pandas as pd
import os, glob
import json
from sklearn.model_selection import train_test_split
import nltk
from os import getcwd
import random

In [2]:
#from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
pip install newspaper3k 

Note: you may need to restart the kernel to use updated packages.


In [4]:
from newspaper import Article

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
#headlines_processed = []
#clean_news = []
text = []
title = []
count = 0

In [8]:
#path_covid_dataset = './dataset/16119_db21c91a1ab47385bb13773ed8238c31'
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'
getcwd()

'/home/ubuntu/w266-finalproject'

In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [10]:
import re
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
#    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    #text = " ".join(text)
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text

In [11]:
import codecs
import io
import time


text = []
title = []
count = 0

X = []
Y = []
Z = []
W = []

start = time.time()

path_cnn_downloads = './dataset/cnn/downloads'
url =  'https://www.google.com/'

filenames = glob.glob(os.path.join(path_cnn_downloads, '*.html'))
#filenames = filenames[0:1]
# Number of files 92579

from joblib import Parallel, delayed

def process_article(filename):
    article_text = ""
    article_title = ""
    article_keywords = ""
    article_summary = ""
    try:
        with io.open(filename, mode="r", encoding="unicode_escape") as f:
            html = f.read()
            html = html.encode('utf-8').strip()
            article = Article(url="%s" % (url), language='en')
            article.download(input_html=html)
            article.parse()
            article.nlp()
            article_text = preprocess(article.text.strip())
            #article_title = article.title.encode('utf-8').strip()
            article_title = preprocess(article.title.strip())
            article_keywords = ' '.join([str(item) for item in article.keywords])
            article_summary = preprocess(article.summary.strip())
            f.close()
    except:
        f.close()
    return article_text, article_title, article_keywords, article_summary

S = Parallel(n_jobs=24)(delayed(process_article)(filename) for filename in filenames)

S = list(filter(None, S))

print("Response time (m): ", round((time.time() - start)/60, 2))


Response time (m):  11.43


In [12]:
S_backup = S
X = []
Y = []
Z = []
W = []
S_ = []
for i in range(len(S)):
    if len(S[i][0]) == 0 or len(S[i][1]) == 0 or len(S[i][2]) == 0 or len(S[i][3]) == 0:
        continue
    else:
        X.append(S[i][0])
        Y.append(S[i][1])
        Z.append(S[i][2])
        W.append(S[i][3])
        S_.append(S[i])

S = S_

In [13]:
count = 0
for i in range(len(S)):
    if len(S[i][0]) == 0 or len(S[i][1]) == 0 or len(S[i][2]) == 0 or len(S[i][3]) == 0:
        count += 1

print(count)

0


In [20]:
S[8901][0]

'  This time Paul Octopus able predict winner .  In World Cup already marked stunning victories catastrophic defeats trying guess next World Cup champion pretty much moot .  Germany riding high annihilating host country Brazil 71 semifinal defeat brutal Brazilian President Dilma Rousseff said would never imagined horror worst nightmares .  Of course Germans would love repeat Argentina .  But anyone show grit calm pressure Argentinians .  Their semifinal Netherlands came heartpounding penalty shootout Argentina 42 victory cemented team first trip World Cup final 24 years .  Germans Argentinians faced World Cup finals twice before .  Back West Germany still country lost Argentina 1986 .  Four years later Argentina lost West Germany 1990 .  Sadly unlike last World Cup 2010 psychic cephalopod placing bets year victor .  In tournament Paul Octopus correctly predicted outcome eight consecutive matches dooming Germany hopes picked Spain semifinal also final .  Paul died German aquarium later 

In [21]:
S[8901][1]

'World Cup final Germany Argentina face battle titans'

In [22]:
S[8901][2]

'argentina titans cup octopus paul west face final germany battle lost world semifinal later'

In [23]:
S[8901][3]

'In World Cup already marked stunning victories catastrophic defeats trying guess next World Cup champion pretty much moot .  Germans Argentinians faced World Cup finals twice before .  Back West Germany still country lost Argentina 1986 .  Four years later Argentina lost West Germany 1990 .  Sadly unlike last World Cup 2010 psychic cephalopod placing bets year victor . '

In [24]:
len(S)

92328

In [25]:
S_train, S_test = train_test_split(S, test_size=0.2)

In [26]:
S_valid, S_test = train_test_split(S_test, test_size=0.5)

In [27]:
X_train = []
Y_train = []
Z_train = []
W_train = []

X_valid = []
Y_valid = []
Z_valid = []
W_valid = []

X_test = []
Y_test = []
Z_test = []
W_test = []

In [28]:
for i in range(len(S_train)):  
        X_train.append(S_train[i][0])
        Y_train.append(S_train[i][1])
        Z_train.append(S_train[i][2])
        W_train.append(S_train[i][3])


In [29]:
for i in range(len(S_valid)):  
        X_valid.append(S_valid[i][0])
        Y_valid.append(S_valid[i][1])
        Z_valid.append(S_valid[i][2])
        W_valid.append(S_valid[i][3])
        
for i in range(len(S_test)):  
        X_test.append(S_test[i][0])
        Y_test.append(S_test[i][1])
        Z_test.append(S_test[i][2])
        W_test.append(S_test[i][3])

In [30]:
# text_train = text[0:len(X_train)]
# title_train = title[0:len(X_train)]

# text_valid = text[len(X_train):len(X_train)+len(X_valid)]
# title_valid = title[len(X_train):len(X_train)+len(X_valid)]

# text_test = text[len(X_train)+len(X_valid):len(X_train)+len(X_valid)+len(X_test)]
# title_test = title[len(X_train)+len(X_valid):len(X_train)+len(X_valid)+len(X_test)]

In [31]:
# print(len(text_train), len(title_train), len(text_valid), len(title_valid), len(text_test), len(title_test))

In [32]:
print(len(X_train), len(Y_train), len(X_valid), len(Y_valid), len(X_test), len(Y_test))

73862 73862 9233 9233 9233 9233


In [33]:

dict_train = {
    'headlines' : Y_train,
    'text' : X_train,
    'keywords' : Z_train,
    'summary_art' : W_train
}

list_to_json_array = json.dumps(dict_train)

jsonString = json.dumps(dict_train)
jsonFile = open(os.path.join(path_processed_data, 'cnn_train.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()



In [34]:
dict_valid = {
    'headlines' : Y_valid,
    'text' : X_valid,
    'keywords' : Z_valid,
    'summary_art' : W_valid
}

list_to_json_array = json.dumps(dict_valid)

jsonString = json.dumps(dict_valid)
jsonFile = open(os.path.join(path_processed_data, 'cnn_valid.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()

In [35]:

# dict_test = {
#     'headlines' : Y_test,
#     'text' : X_test,
#     'keywords' : Z_test,
#     'summary_art' : W_test
# }

# list_to_json_array = json.dumps(dict_test)

# jsonString = json.dumps(dict_test)
# jsonFile = open(os.path.join(path_processed_data, 'cnn_test.json'), "w")
# jsonFile.write(jsonString)
# jsonFile.close()

In [36]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
S1s = []
S3s = []
for item in X_test:
    sentences = nltk.sent_tokenize(item)
    len_sentences = min(3, len(sentences))
    S3s.append(' '.join(sentences[:len_sentences]))
    S1s.append(' '.join(sentences[:1]))

print(len(S1s))

9233


In [38]:
print(len(S3s))

9233


In [39]:
S3s

['NEW YORK   Dominick Dunne former Hollywood producer bestselling author known Vanity Fair essays courtroom travails rich famous died Wednesday New York city long battle bladder cancer . Dominick Dunne wrote five best selling books covered high society crimes Vanity Fair . Dunne described a highclass Zelig 83 .',
 ' take hand cause tramps  like us baby born run . On December 5 almost 40 years later handwritten genesis one The Boss biggest hits  one rock n roll wellknown songs  auctioned highest bidder . Sotheby New York expects manuscript features 30 lines marginal notations scrawled blue ink fetch 70000 100000 .',
 'Los Angeles California   Volunteers flocked southern California community help residents mop last week flood damage ahead meteorologists predicting rain . Five homes destroyed nearly 75 damaged San Bernardino County community Highland water mud rushed foothill town last week authorities said . Total damage including cost cleanup lost property value could hit 17 .',
 'Edito

In [40]:
# Another approach to get sentence tokens
# S1s = []
# S3s = []
# for item in X_test:
#     sentences = re.split(r'(?<=[.:;])\s', item)
#     len_sentences = min(3, len(sentences))
#     S3s.append(' '.join(sentences[:len_sentences]))
#     S1s.append(' '.join(sentences[:1]))

# print(len(S1s))

In [41]:
Z_test

['dunne writer dies trial simpson known fair magazine life dominick wrote society vanity crime york 83',
 'auction genesis sothebys songs handwritten rock born spector run thought springsteen young springsteens block trap hit',
 'week county storm rushes water volunteers mud southern damage ahead flood community san help mop california',
 'ticket workers voters state ohio republicans winner ohioan portman presidential republican buckeye',
 'student curriculum transcript daily cnn todays think 26 standards page world',
 'copahue issued border volcanism chileargentina significant kilometers eruption service andes alert miles volcano red',
 'human arrested ousted family issues ben president warrant leader tap rights ali tunisia members',
 'denies latest supreme revive california samesex gay court bid state legal officials ban marriage ruling',
 'sell gifts bid glove wore signed julien jackson auction michael orange music world heading',
 'scholars tohti xinjiang possible economic rights r

In [42]:
# To extract the NEs
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [43]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [46]:
from collections import Counter
 
def remove_duplicates(input):
 
    # split input string separated by space
    input = input.split(" ")
 
    # now create dictionary using counter method
    # which will have strings as key and their
    # frequencies as value
    unique_words = Counter(input)
 
    # joins two adjacent elements in iterable way
    s = " ".join(unique_words.keys())
    return s

In [51]:
# example = "My name is Wolfgang and I live in Berlin, Saratoga, December 2019, Washington, Bill, Delhi, Modi Putin "

# ner_results = nlp(example)
# print(ner_results)

entities = []
ner = []
for item in X_test:
    ner_results = nlp(item)
    ner.append(ner_results)
    nes = []
    for ne in range(len(ner_results)):
        if ner_results[ne]['entity'] == 'B-PER' or ner_results[ne]['entity'] == 'I-PER' or ner_results[ne]['entity'] == 'B-LOC' or ner_results[ne]['entity'] == 'I-LOC' or ner_results[ne]['entity'] == 'I-ORG' or ner_results[ne]['entity'] == 'B-ORG' or ner_results[ne]['entity'] == 'I-MISC':
            nes.append(ner_results[ne]['word'])

    if len(nes) == 0:
        nes.append('')
    
    named_entities = remove_duplicates(' '.join(nes).replace(" ##", "").replace("##", ""))
    #named_entities = re.sub(r'\ .* \ ', ' ', named_entities)
    entities.append(named_entities)
            
            
print(len(entities))

9233


In [57]:
import numpy as np
len(np.array(ner_results))

56

In [ ]:
# for item in range(len(ner_results)):
#       if ner_results[item]['entity'] == 'B-PER' or 
#         if ner_results[item]['entity'] == 'I-PER' or 
#         if ner_results[item]['entity'] == 'B-LOC' or
#         if ner_results[item]['entity'] == 'I-LOC'
#             print(ner_results[item]['word'])

In [53]:

size = len(X_test)

# S1s = [''] * size
# S3s = [''] * size
S2 = [''] * size
S3 = [''] * size
S4 = [''] * size
S5 = [''] * size
P0 = [''] * size
P1 = [''] * size
P2 = [''] * size
P3 = [''] * size
P4 = [''] * size
P5 = [''] * size
P6 = [''] * size

dict_test = {
    'headlines' : Y_test,
    'text' : X_test,
    'keywords' : Z_test,
    'summary_art' : W_test,
    'entities' : entities,
#    'ner' : ner_results,
    'sentence_1s' : S1s,
    'sentence_3s' : S3s,
    'summary_ext' : S2,
    'summary_abs' : S3,
    'summary_extabs' : S4,
    'summary_ner' : S5,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6
}

df_test = pd.DataFrame(dict_test)

jsonString = json.dumps(dict_test)
jsonFile = open(os.path.join(path_processed_data, 'cnn_test.json_v3'), "w")
jsonFile.write(jsonString)
jsonFile.close()

In [91]:
df_test

,headlines,text,keywords,summary_art,entities,sentence_1s,sentence_3s,summary_ext,summary_abs,summary_extabs,summary_ner,predict_1s,predict_3s,predict_text,predict_ext,predict_abs,predict_extabs,predict_ner
0,Female WWII pilots honored float Rose Parade,As young woman growing miles OHare Airport o...,rodgers program pilots women military wwii flo...,She trained sunup sundown seven days week Aven...,OHare Airport Chicago Lorraine Rodgers Amelia ...,As young woman growing miles OHare Airport o...,As young woman growing miles OHare Airport o...,,,,,,,,,,,
1,Webb Simpson wins U . S . Open,American golfer Webb Simpson made furious ch...,tournament woods win shot wins open lead simps...,American golfer Webb Simpson made furious ch...,Webb Simpson U . S Bubba Watson Keegan Bradley...,American golfer Webb Simpson made furious ch...,American golfer Webb Simpson made furious ch...,,,,,,,,,,,
2,Football fans battle club ownership,LONDON England say club year could vote decis...,ownership fans supporters club fan liverpool b...,LONDON England Football fans UK ushering pot...,OND England David Davis Walters Davisfle Walte...,LONDON England say club year could vote decis...,LONDON England say club year could vote decis...,,,,,,,,,,,
3,Texas state board says arson investigators use...,A Texas state board said Friday arson invest...,willinghams work investigators texas state wil...,A Texas state board said Friday arson invest...,Texas Cameron Todd Willingham Will Rick Perry ...,A Texas state board said Friday arson invest...,A Texas state board said Friday arson invest...,,,,,,,,,,,
4,Sarah Palin publishing political worlds collision,Editor note Mary Matalin joined CNN Republican...,palins collision palin publishing campaign num...,In today political market well officially rele...,Mary Matalin Mata Ronald Reagan George H . W B...,Editor note Mary Matalin joined CNN Republican...,Editor note Mary Matalin joined CNN Republican...,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9228,Haiti awash doctors nurses short supply,PortauPrince Haiti So many doctors answering...,doctors help general need nurses portauprince ...,iReport Looking loved ones HaitiThe general ho...,Haiti Sanjay Gupta Paul Auer Port,PortauPrince Haiti So many doctors answering...,PortauPrince Haiti So many doctors answering...,,,,,,,,,,,
9229,Skypods Are gondolas next big thing urban tran...,parts world like across shipping lanes fact q...,urban mile million gondolas big line city tran...,area city . Socioeconomic benefitsThese sent...,Big,parts world like across shipping lanes fact q...,parts world like across shipping lanes fact q...,,,,,,,,,,,
9230,How North Korean defector story unraveled,Story highlights Shin Donghyuk made sanitized ...,version harden korean unraveled prison korea n...,Shortly escaping North Korea Shin Donghyuk for...,Shin Donghy North Korea Blaine Hard China,Story highlights Shin Donghyuk made sanitized ...,Story highlights Shin Donghyuk made sanitized ...,,,,,,,,,,,
9231,IsraeliPalestinian relations hit new snags Ker...,Efforts forge lasting peace Israelis Palesti...,palestinian relations palestinians west kerry ...,Kerry met Israeli Prime Minister Benjamin Neta...,U . S John Kerry Middle East Benjamin Netanyah...,Efforts forge lasting peace Israelis Palesti...,Efforts forge lasting peace Israelis Palesti...,,,,,,,,,,,


In [92]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_train = details_dict['text']
Y_train = details_dict['headlines']
Z_train = details_dict['keywords']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_valid = details_dict['text']
Y_valid = details_dict['headlines']
Z_valid = details_dict['keywords']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text']
Y_test = details_dict['headlines']
Z_test = details_dict['keywords']

In [93]:
X_test[11:12]

['Atlanta   Atlanta police issued arrest warrant rapper Gucci Mane allegedly struck man head champagne bottle downtown Atlanta nightclub .  The man told police tried begin conversation rapper Harlem Nights Ultra Lounge club suddenly struck .  The attack left deep gash man head Atlanta police Sgt .  Greg Lyon said .  Im military .  I wanted get picture Gucci Mane man told CNN affiliate WSBTV .  I speaking security guard Gucci Mane hit head bottle .  The man affiliate identified James said second man hit side face .  The incident took place March 16 VIP section club mixtape release party .  The 33yearold rapper whose real name Radric Davis wanted aggravated assault .  His reps immediately available comment . ']

In [94]:
Y_test[11:12]

['Atlanta police issue arrest warrant rapper Gucci Mane nightclub incident']

In [95]:
Z_test[11:12]

['rapper arrest cnn head hit incident wanted gucci issue nightclub told atlanta mane man warrant']